## Model Context Protocol (MCP)

### Background - why MCP rises?

In our previous [function calling notebook](function_calling.ipynb), we learnt how to **use external tools** to enable LLMs to perform more complicated tasks based on OpenAI API. 

As different model provider arises, there are *not a universal standard / format for managing function calling* for different LLM APIs. For a quick example, OpenAI return the tool use request as a `"type": "function_call"` message ([source](https://platform.openai.com/docs/guides/function-calling?api-mode=responses#handling-function-calls)), while Claude returns a `"type": "tool_use"` message ([source](https://docs.anthropic.com/en/api/messages#body-tools)).

Besides the formatting issue, function calling also assumes the tools are runned locally, together with where the chat requests were sent.

To unify these inconsistences and provide external tool usage, MCP come into play. As in the [Claude MCP](https://modelcontextprotocol.io/docs/getting-started/intro) states:

> MCP is an open protocol that **standardizes how applications provide context to LLMs**,..., a standardized way to connect AI models to different data sources and tools.

### What are MCP components?

> MCP follows a client-server architecture where an MCP host — an AI application like Claude Code or Claude Desktop — establishes connections to one or more MCP servers. The MCP host accomplishes this by creating one MCP client for each MCP server. Each MCP client maintains a dedicated one-to-one connection with its corresponding MCP server.
>
> The key participants in the MCP architecture are:
> - MCP Host: The AI application that coordinates and manages one or multiple MCP clients
> - MCP Client: A component that maintains a connection to an MCP server and obtains context from an MCP server for the MCP host to use
> - MCP Server: A program that provides context to MCP clients

The official explaination may seem vague, let's translate them into plain English (with a little lost in precision):

MCP defines the behavior between a client and a server like how we are surfing the web. 
- A MCP server (corresponds to content server) provide tool/resource/prompt services.
- A MCP client (corresponds to a subprocess in your browser, example subprocess:render image / render table) are used to send tool calling / resource acquisition requests to server.
- A MCP host (corresponds to your browser) is the application that manages all clients.



## Build a simple MCP server and run them in Claude Desktop

Now let's set up a toy mcp server at local following [Claude's tutorial](https://modelcontextprotocol.io/quickstart/server#python).

### Reuse code at toy_projects/weather_mcp

In [ ]:
cd toy_projects/weather_mcp

# activate environment and install dependencies
uv venv
source .venv/bin/activate
uv add "mcp[cli]" httpx

# run the server
uv run weather.py

### Build from scratch

#### 1. Initialize environment

In [ ]:
# Create a new directory for our project
uv init weather_mcp
cd weather_mcp

# Create virtual environment and activate it
uv venv
source .venv/bin/activate

# Install dependencies
uv add "mcp[cli]" httpx

# Create our server file
touch weather.py

#### 2. Write server code

Things need to do:
1. setting up server instance

    ```python
    from mcp.server.fastmcp import FastMCP
    mcp = FastMCP("mcp_name_here")
    ```

2. Implement tools and register it with `@mcp.tool()`
    ```python
    @mcp.tool()
    async def get_alerts(state: str) -> str:
        ...some funtioning code
    ```

3. run the server in backend
    ```python
    if __name__ == "__main__":
        mcp.run(transport='stdio')
    ```

For detailed code, check [this python file](toy_projects/weather_mcp/weather.py). A simple code scaffold could be found below.

In [ ]:
from typing import Any
from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("mcp_name_here")

# Helper functions for tools
def make_nws_request(url: str) -> dict[str, Any] | None:
    pass

# Define tools
@mcp.tool()
async def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    pass

if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

#### 3. Register in Claude desktop

Add your servers in the mcpServers key, configure stores in `code ~/Library/Application\ Support/Claude/claude_desktop_config.json`.

use `which uv` to get the full path.

```json
{
  "mcpServers": {
    "weather": {
      "command": "/full/path/to/uv",
      "args": [
        "--directory",
        "/ABSOLUTE/PATH/TO/PARENT/FOLDER/weather",
        "run",
        "weather.py"
      ]
    }
  }
}
```

Then reload your Claude desktop, you should see the mcp there.

<img src="assets/imgs/claude_mcp_example.jpg" alt="locating-mcp-tools" style="width:40%;"/>

Then we can use mcp based on our query.

<img src="assets/imgs/claude_mcp_result.png" alt="check-mcp-records" style="width:40%;"/>